# chatbots
- https://python.langchain.com/docs/tutorials/chatbot/

In [1]:
# llm模型設定
# https://build.nvidia.com/deepseek-ai/deepseek-r1
# nvapi-xxx
import getpass
import os
if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

Enter API key for NVIDIA:  ········


In [2]:
!uv pip install -qU "langchain-nvidia-ai-endpoints"

In [2]:
## 模型來源
# https://build.nvidia.com/deepseek-ai
# https://build.nvidia.com/meta
# https://build.nvidia.com/google
# https://build.nvidia.com/qwen

from langchain.chat_models import init_chat_model
llm = init_chat_model("meta/llama-4-maverick-17b-128e-instruct", model_provider="nvidia")

D:\uv\langchaing\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:212: UserWarning: Found meta/llama-4-maverick-17b-128e-instruct in available_models, but type is unknown and inference may fail.
  warnings.warn(


In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("You are a ai assistant"),
    HumanMessage("Hi! I'm Bob!"),
]

response=llm.invoke(messages)
print(response)
print("---------------")
print(response.content)

content="Nice to meet you, Bob! How's your day going so far?" additional_kwargs={} response_metadata={'role': 'assistant', 'reasoning_content': None, 'content': "Nice to meet you, Bob! How's your day going so far?", 'tool_calls': [], 'token_usage': {'prompt_tokens': 25, 'total_tokens': 41, 'completion_tokens': 16, 'prompt_tokens_details': None}, 'finish_reason': 'stop', 'model_name': 'meta/llama-4-maverick-17b-128e-instruct'} id='run--8c8c5ab2-767b-405e-a098-0ce96acc6221-0' usage_metadata={'input_tokens': 25, 'output_tokens': 16, 'total_tokens': 41} role='assistant'
---------------
Nice to meet you, Bob! How's your day going so far?


In [4]:
llm.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="You haven't told me your name, so I don't know it. I'm happy to chat with you and learn more about you, though! Would you like to share your name with me?", additional_kwargs={}, response_metadata={'role': 'assistant', 'reasoning_content': None, 'content': "You haven't told me your name, so I don't know it. I'm happy to chat with you and learn more about you, though! Would you like to share your name with me?", 'tool_calls': [], 'token_usage': {'prompt_tokens': 14, 'total_tokens': 52, 'completion_tokens': 38, 'prompt_tokens_details': None}, 'finish_reason': 'stop', 'model_name': 'meta/llama-4-maverick-17b-128e-instruct'}, id='run--f8bd422e-1e5e-4ef8-9d73-42c9ad365057-0', usage_metadata={'input_tokens': 14, 'output_tokens': 38, 'total_tokens': 52}, role='assistant')

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

llm.invoke(
    [
        SystemMessage("You are a ai assistant"),
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! You told me that when you first said hello.', additional_kwargs={}, response_metadata={'role': 'assistant', 'reasoning_content': None, 'content': 'Your name is Bob! You told me that when you first said hello.', 'tool_calls': [], 'token_usage': {'prompt_tokens': 48, 'total_tokens': 64, 'completion_tokens': 16, 'prompt_tokens_details': None}, 'finish_reason': 'stop', 'model_name': 'meta/llama-4-maverick-17b-128e-instruct'}, id='run--d268f159-f1a9-4145-824e-389a70bf474a-0', usage_metadata={'input_tokens': 48, 'output_tokens': 16, 'total_tokens': 64}, role='assistant')

## LangGraph實現了內建持久層，使其成為支援多輪對話的聊天應用程式的理想選擇。

In [8]:
# 匯入必要模組
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# 建立一個新的語言模型工作流程（Graph），使用 MessagesState 定義狀態結構（通常為多輪訊息列表）
workflow = StateGraph(state_schema=MessagesState)


# 定義模型處理節點的邏輯函式
def call_model(state: MessagesState):
    # 從狀態中取得歷史訊息（message history）
    # 並呼叫語言模型進行處理（這裡的 llm 須事先定義，例如：ChatOpenAI(...)）
    response = llm.invoke(state["messages"])
    # 回傳新的狀態，仍然是以 messages 為主
    return {"messages": response}


# 在流程中添加節點與邊（edge）：
# 1. 從 START 開始，轉到 "model" 節點
workflow.add_edge(START, "model")

# 2. 定義 "model" 節點的邏輯為 call_model 函式
workflow.add_node("model", call_model)


# 建立記憶體儲存器，用於保存流程中的狀態資料
memory = MemorySaver()

# 編譯整個工作流程為可執行應用（App），並指定使用記憶體儲存
app = workflow.compile(checkpointer=memory)

# 範例：如果你想要執行這個流程，可使用 app.invoke()
#config = {"configurable": {"thread_id": "abc123"}}
#query = "Hi! I'm Bob."
#input_messages = [HumanMessage(query)]
#result = app.invoke({"messages": input_messages}, config)
#print(result["messages"])

In [9]:
config = {"configurable": {"thread_id": "abc123"}}
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Nice to meet you, Bob! How's it going?


In [10]:
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! I remember, you told me just a moment ago.


In [11]:
# 更換 thread_id
config = {"configurable": {"thread_id": "abc234"}}
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You haven't told me your name, so I don't know it. Would you like to share it with me?


In [12]:
# 更換 thread_id
config = {"configurable": {"thread_id": "abc123"}}
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me your name is Bob. Is that correct?


## 習題

**擴展聊天機器人功能**

1.  **基本功能：**
    * 在 notebook 中，我們創建了一個基本的聊天機器人，它可以記住用戶的名字並在同一 session 中使用它。
    * 你的任務是擴展這個聊天機器人，讓它可以記住多個使用者的資訊，並且在不同的 session 中也能夠正確辨識使用者。

2.  **情境設定：**
    * 將聊天機器人設定為一個線上書店的客服機器人。
    * 除了基本的問候和回答問題外，聊天機器人還需要能夠處理以下功能：
        * 查詢書籍資訊（例如：書名、作者、價格、庫存）。
        * 提供書籍推薦。
        * 處理訂單查詢（例如：訂單狀態、物流資訊）。


**評估標準：**

* 程式碼是否能夠正確區分和辨識不同使用者？ (30%)
* 使用者資訊是否能夠持久化儲存，並在不同 session 中正確載入？ (30%)
* 聊天機器人是否能夠完成指定的情境任務（查詢書籍、提供推薦、處理訂單）？ (40%)


